In [1]:
import tensorflow as tf
import numpy as np
from scipy.io import wavfile
from scipy.signal import spectrogram, stft, istft
import matplotlib.pyplot as plt
import librosa
import librosa.display
from misceallaneous import getWavFileAsNpArray, displaySpectrogram
from IPython.display import Audio
import kapre

samplerate = 12000
nperseg = 1024

clean = getWavFileAsNpArray("../dataset_2/clean/p1.wav")
white = getWavFileAsNpArray("../dataset_2/white/p1.wav")
clean = np.array(clean, dtype="int32")
white = np.array(white, dtype="int32")

clean_split = []
white_split = []

samples_length = nperseg

for i in range(0, clean.shape[0]-samples_length, samples_length):
    clean_split.append(clean[i:i+samples_length])
    white_split.append(white[i:i+samples_length])
clean_split = np.array(clean_split)
white_split = np.array(white_split)

print(clean_split.shape)
print(white_split.shape)

(10659, 1024)
(10659, 1024)


@inproceedings{choi2017kapre,
  title={Kapre: On-GPU Audio Preprocessing Layers for a Quick Implementation of Deep Neural Network Models with Keras},
  author={Choi, Keunwoo and Joo, Deokjin and Kim, Juho},
  booktitle={Machine Learning for Music Discovery Workshop at 34th International Conference on Machine Learning},
  year={2017},
  organization={ICML}
}

In [2]:
def process_stft(inputs):
    return tf.signal.stft(inputs, frame_length=nperseg, frame_step=nperseg)

In [9]:
def classifier(input_shape):
    inputs = tf.keras.Input(shape=input_shape)
    x0 = tf.keras.layers.Lambda(process_stft)(inputs)
    x1 = tf.keras.layers.Dense(512, activation="tanh")(x0)
    x3 = tf.keras.layers.Dense(100, activation="tanh")(x1)
    outputs = tf.keras.layers.Dense(1, activation="sigmoid")(x3)
    model = tf.keras.Model(inputs=inputs, outputs=outputs, name="discriminator")
    model.summary()
    model.compile(optimizer= 'adam', loss='mse', metrics=['accuracy'])
    return model

d = classifier(nperseg)

Model: "discriminator"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         [(None, 1024)]            0         
_________________________________________________________________
lambda_3 (Lambda)            (None, 1, 513)            0         
_________________________________________________________________
dense_12 (Dense)             (None, 1, 512)            263168    
_________________________________________________________________
dense_13 (Dense)             (None, 1, 100)            51300     
_________________________________________________________________
dense_14 (Dense)             (None, 1, 1)              101       
Total params: 314,569
Trainable params: 314,569
Non-trainable params: 0
_________________________________________________________________


In [10]:
clean_labels = np.ones((clean_split.shape[0], 1))
white_labels = np.zeros((white_split.shape[0], 1))

for j in range(50):
    d.fit(np.concatenate((clean_split, white_split)), np.concatenate((clean_labels, white_labels)))

336/667 [==============>...............] - ETA: 1s - loss: 0.2311 - accuracy: 0.5996

KeyboardInterrupt: 

In [ ]:
i = [np.reshape(white_split[k, :], (1, white_split.shape[1])) for k in range(25)]
o = [g.predict(i[k]) for k in range(len(i))]
p = [np.reshape(i[k].T, (white_split.shape[1],)) for k in range(len(o))]
q = [np.reshape(o[k].T, (white_split.shape[1],)) for k in range(len(o))]


a = np.concatenate(p)
b = np.concatenate(q)
Audio(a, rate=samplerate)

In [ ]:
Audio(b, rate=samplerate)